In [1]:
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt
import scipy
import os
from pathlib import Path
import math
from statsmodels.stats.multitest import multipletests
import statsmodels
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection as fdr

In [2]:
deg_h =pd.read_csv('healthy_segnificant.txt', sep="\t", header=0)
deg_c =pd.read_csv('cancerous_segnificant.txt', sep="\t", header=0)
print(len(deg_h), len(deg_c))
print(deg_h.shape, deg_c.dropna().shape)
print(deg_h.columns, deg_c.columns)

8595 8595
(8595, 54) (8595, 54)
Index(['Unnamed: 0', 'Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657',
       'TCGA-58-8386', 'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670',
       'TCGA-60-2709', 'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471',
       'TCGA-22-4609', 'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483',
       'TCGA-56-8623', 'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394',
       'TCGA-34-8454', 'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773',
       'TCGA-51-4080', 'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771',
       'TCGA-92-7340', 'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823',
       'TCGA-22-5491', 'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008',
       'TCGA-56-7730', 'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731',
       'TCGA-77-7337', 'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710',
       'TCGA-56-7580', 'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083',
       'TCGA-51-4079', 'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081',
       'TCGA-56-8309', 'average_GE'],
      dt

In [3]:
deg_h.columns[3:-1].shape

(50,)

In [4]:
print((deg_h.columns == deg_c.columns).all())

True


In [5]:
h_cols = []
for column in deg_h.columns[3:-1]:
    h_cols.append(column + '_h')
# h_cols

In [6]:
c_cols = []
for column in deg_c.columns[3:-1]:
    c_cols.append(column + '_c')
# c_cols

In [7]:
new = h_cols + c_cols
cols = list(deg_h.columns[3:-1]) + list(deg_c.columns[3:-1])
print(np.array(new).shape)
np.array(cols).shape


(100,)


(100,)

In [8]:
h_names = {list(deg_h.columns[3:-1])[i]: h_cols[i] for i in range(len(h_cols))}
c_names = {list(deg_c.columns[3:-1])[i]: c_cols[i] for i in range(len(c_cols))}

# h_names
# c_names

In [9]:
deg_h.rename(h_names, axis=1, inplace=True)
deg_c.rename(c_names, axis=1, inplace=True)

# deg_h
# deg_c

In [10]:
gct = pd.concat([deg_h.iloc[:,3:-1], deg_c.iloc[:,3:-1]], axis = 1)

In [11]:
gct.columns

Index(['TCGA-43-7657_h', 'TCGA-58-8386_h', 'TCGA-22-5478_h', 'TCGA-22-5472_h',
       'TCGA-43-5670_h', 'TCGA-60-2709_h', 'TCGA-22-5489_h', 'TCGA-77-8007_h',
       'TCGA-22-5471_h', 'TCGA-22-4609_h', 'TCGA-22-5482_h', 'TCGA-56-8082_h',
       'TCGA-22-5483_h', 'TCGA-56-8623_h', 'TCGA-33-4587_h', 'TCGA-56-7579_h',
       'TCGA-43-3394_h', 'TCGA-34-8454_h', 'TCGA-77-7338_h', 'TCGA-43-6143_h',
       'TCGA-43-6773_h', 'TCGA-51-4080_h', 'TCGA-34-7107_h', 'TCGA-39-5040_h',
       'TCGA-43-6771_h', 'TCGA-92-7340_h', 'TCGA-77-7138_h', 'TCGA-77-7142_h',
       'TCGA-56-7823_h', 'TCGA-22-5491_h', 'TCGA-22-5481_h', 'TCGA-90-7767_h',
       'TCGA-77-8008_h', 'TCGA-56-7730_h', 'TCGA-33-6737_h', 'TCGA-56-7582_h',
       'TCGA-56-7731_h', 'TCGA-77-7337_h', 'TCGA-43-7658_h', 'TCGA-56-8201_h',
       'TCGA-85-7710_h', 'TCGA-56-7580_h', 'TCGA-43-6647_h', 'TCGA-90-6837_h',
       'TCGA-56-8083_h', 'TCGA-51-4079_h', 'TCGA-56-7222_h', 'TCGA-22-4593_h',
       'TCGA-51-4081_h', 'TCGA-56-8309_h', 'TCGA-43-

In [12]:
deg_h.Hugo_Symbol.shape

(8595,)

In [13]:
gct['NAME'] = deg_h.Hugo_Symbol
gct['DESCRIPTION'] = 'na'

In [14]:
cols = gct.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
cols

['NAME',
 'DESCRIPTION',
 'TCGA-43-7657_h',
 'TCGA-58-8386_h',
 'TCGA-22-5478_h',
 'TCGA-22-5472_h',
 'TCGA-43-5670_h',
 'TCGA-60-2709_h',
 'TCGA-22-5489_h',
 'TCGA-77-8007_h',
 'TCGA-22-5471_h',
 'TCGA-22-4609_h',
 'TCGA-22-5482_h',
 'TCGA-56-8082_h',
 'TCGA-22-5483_h',
 'TCGA-56-8623_h',
 'TCGA-33-4587_h',
 'TCGA-56-7579_h',
 'TCGA-43-3394_h',
 'TCGA-34-8454_h',
 'TCGA-77-7338_h',
 'TCGA-43-6143_h',
 'TCGA-43-6773_h',
 'TCGA-51-4080_h',
 'TCGA-34-7107_h',
 'TCGA-39-5040_h',
 'TCGA-43-6771_h',
 'TCGA-92-7340_h',
 'TCGA-77-7138_h',
 'TCGA-77-7142_h',
 'TCGA-56-7823_h',
 'TCGA-22-5491_h',
 'TCGA-22-5481_h',
 'TCGA-90-7767_h',
 'TCGA-77-8008_h',
 'TCGA-56-7730_h',
 'TCGA-33-6737_h',
 'TCGA-56-7582_h',
 'TCGA-56-7731_h',
 'TCGA-77-7337_h',
 'TCGA-43-7658_h',
 'TCGA-56-8201_h',
 'TCGA-85-7710_h',
 'TCGA-56-7580_h',
 'TCGA-43-6647_h',
 'TCGA-90-6837_h',
 'TCGA-56-8083_h',
 'TCGA-51-4079_h',
 'TCGA-56-7222_h',
 'TCGA-22-4593_h',
 'TCGA-51-4081_h',
 'TCGA-56-8309_h',
 'TCGA-43-7657_c',
 'TCGA

In [15]:
gct = gct[cols]
gct

,NAME,DESCRIPTION,TCGA-43-7657_h,TCGA-58-8386_h,TCGA-22-5478_h,TCGA-22-5472_h,TCGA-43-5670_h,TCGA-60-2709_h,TCGA-22-5489_h,TCGA-77-8007_h,...,TCGA-85-7710_c,TCGA-56-7580_c,TCGA-43-6647_c,TCGA-90-6837_c,TCGA-56-8083_c,TCGA-51-4079_c,TCGA-56-7222_c,TCGA-22-4593_c,TCGA-51-4081_c,TCGA-56-8309_c
0,HIST3H2A,na,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LXN,na,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
2,CNKSR2,na,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
3,AKR1C1,na,1088.92,947.83,684.02,860.08,1096.50,2090.03,1369.04,1151.06,...,884.29,1175.27,1143.10,656.11,334.46,52135.28,2133.97,934.76,37379.55,5403.70
4,C3orf62,na,122.64,181.28,181.28,150.17,183.82,117.60,106.63,150.17,...,62.56,129.69,48.18,167.90,80.01,65.26,104.42,68.55,122.64,84.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8590,ZNF521,na,215.77,148.09,83.45,232.94,167.90,80.01,161.02,104.42,...,22.59,33.78,148.09,106.63,297.17,22.26,25.72,106.63,71.00,37.05
8591,SPINT2,na,6792.79,5441.30,5831.91,5329.30,5711.87,6164.49,8134.41,6516.03,...,18052.61,15392.14,20030.74,20593.91,9808.75,11189.60,10733.74,10014.87,6983.79,23169.48
8592,HAVCR2,na,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
8593,CTD-2116N17.1,na,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [16]:
gct.to_csv('GCT.txt', sep="\t", index=False)